# Imports

In [2]:
import pandas as pd
import os
import string
from itertools import groupby, product, combinations, permutations
from heapq import nlargest
from functools import lru_cache, cmp_to_key, wraps
import re
import random
import numpy as np
from copy import deepcopy
import pprint
pp = pprint.PrettyPrinter(indent=2)
import networkx as nx
from collections import defaultdict, deque, namedtuple
from datetime import datetime
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import z3
from frozendict import frozendict

In [21]:
def open_source(day):
    with open(f'source/day{day}.txt', 'r') as f:
        lista = f.readlines()
    lista = [x.strip('\n') for x in lista]
    return lista

def freezeargs(func):
    """Transform mutable dictionnary into immutable. Compatible with cache."""

    @wraps(func)
    def wrapped(*args, **kwargs):
        args = tuple([frozendict(arg) if isinstance(arg, dict) else arg for arg in args])
        kwargs = {k: frozendict(v) if isinstance(v, dict) else v for k, v in kwargs.items()}
        return func(*args, **kwargs)
    return wrapped

class HDict(dict):
    def __hash__(self):
        return hash(frozenset(self.items()))

In [128]:
path = '/Users/barnabas.kadar/Own_Repos/advent/2022/source'

def create_txt_files(path):
    files = [f"day{x}" if x > 9 else f"day0{x}" for x in range(1, 26)]
    for file in files:
        try:
            with open(os.path.join(path, file + ".txt"), 'x') as fp:
                pass
            with open(os.path.join(path, file + "_example.txt"), 'x') as fp:
                pass
        except FileExistsError:
            print("file existed but continued loop")
            continue

# Day 1

In [171]:
lista = open_source("01")

In [183]:
result = [sum([int(x) for x in a[1]]) for a in groupby(lista, key=lambda s: not s) if not a[0]]

In [184]:
max(result)

71780

In [187]:
sum(nlargest(3, result))

212489

# Day 2

In [71]:
lista = open_source("02")

In [72]:
scoring1 = {
    'A X': 4,
    'A Y': 8,
    'A Z': 3,
    'B X': 1,
    'B Y': 5,
    'B Z': 9,
    'C X': 7,
    'C Y': 2,
    'C Z': 6
}

In [73]:
scoring2 = {
    'A X': 3,
    'A Y': 4,
    'A Z': 8,
    'B X': 1,
    'B Y': 5,
    'B Z': 9,
    'C X': 2,
    'C Y': 6,
    'C Z': 7
}

In [74]:
sum([scoring1[x] for x in lista])

10624

In [75]:
sum([scoring2[x] for x in lista])

14060

# Day 3

In [4]:
lista = open_source("03")

In [5]:
scoring = dict(zip(list(string.ascii_lowercase) + list(string.ascii_uppercase), list(range(1, 53))))

In [26]:
total = 0
for sack in lista:
    half = len(sack)//2
    first, second = set(sack[half:]), set(sack[:half])
    inter = set.intersection(first, second).pop()
    total += scoring[inter]
total

7821

In [7]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [25]:
total = 0
for sack in chunks(lista, 3):
    inter = set.intersection(*list(map(lambda x: set(x), sack)))
    total += scoring[inter.pop()]
total

2752

# Day 4

In [5]:
lista = open_source('04')

In [10]:
count1 = count2 = 0
for task in lista:
    fs, fe, ss, se = list(map(int, re.split(',|-', task)))
    if (fs >= ss and fe <= se) or (fs <= ss and fe >= se):
        count1 += 1
    if fs <= se and ss <= fe:
        count2 += 1
print(count1, count2)

513 878


# Day 5

In [438]:
lista = open_source('05')

In [439]:
def convert_day5_input(lista):
    stacks = []
    proc = []
    stacks_dict = {}
    for sor in lista:
        if len(sor) > 20:
            stacks.append(list(sor[1::4]))
        elif 'move' in sor:
            proc.append(re.findall('\d+', sor))
    stacks = np.rot90(stacks, 3)
    for stack in stacks:
        k, *v = [x for x in stack if x != ' ']
        stacks_dict[k] = v
    return stacks_dict, proc

def get_day5(s, proc, part_1=True):
    stacks = deepcopy(s)
    for qty, old, new in proc:
        qty = int(qty)
        if part_1:
            for _ in range(qty):
                stacks[new].append(stacks[old].pop(-1))
        else:
            stacks[new] += stacks[old][-qty:]
            del stacks[old][-qty:]
    return ''.join([x[-1] for x in stacks.values()])

In [440]:
s, p = convert_day5_input(lista)

In [441]:
get_day5(s, p, part_1=True)

'TPGVQPFDH'

In [442]:
get_day5(s, p, part_1=False)

'DMRDFRHHH'

# Day 6

In [58]:
lista = open_source('06')

In [61]:
buffer = 14
for elem in lista:
    for i, c in enumerate(elem):
        result = elem[i:i + buffer]
        if len(set(result)) == buffer:
            print(i + buffer)
            break

3534


# Day 7

In [128]:
lista = open_source('07')

In [146]:
path = []
files = defaultdict(int)
for l in lista:
    if l.startswith('$'):
        if ' cd ' in l:
            if l.endswith('..'):
                path.pop()
            else:
                path.append(l.split()[-1])
    elif not l.startswith('dir'):
        for i, _ in enumerate(path):
            files['|'.join(path[:i + 1])] += int(l.split()[0])
            
sum(x for x in files.values() if x <= 100000)

1444896

In [147]:
min(x for x in files.values() if x >= files['/'] - 40000000)

404395

# Day 8

In [5]:
lista = open_source('08')
lista = np.array([[int(x) for x in list(x)] for x in lista])

In [15]:
def visibl(part):
    return all(np.greater(C, part))

def view(part, reverse=False):
    if reverse:
        part = np.flip(part)
    result = np.argmax(np.append(part, 9) >= C)
    if len(part) != result:
        result += 1
    return result

In [20]:
visible = 4 * (len(lista) - 1)
max_scenic = 0
R = range(1, len(lista) - 1)
for x, y in product(R, R):
    C, l, u, r, d = lista[x, y], lista[x, :y], lista[:x, y], lista[x, y + 1:], lista[x + 1:, y]
    
    if visibl(l) or visibl(u) or visibl(r) or visibl(d):
        visible += 1
        
    scenic = view(l, True) * view(u, True) * view(r) * view(d)
    if max_scenic < scenic:
        max_scenic = scenic
print(f"part1: {visible}\npart2: {max_res}")

part1: 1851
part2: 574080


# Day 9

In [664]:
lista = open_source('09')

In [665]:
dirs = dict(zip(['R', 'U', 'L', 'D'], [(1, 0), (0, 1), (-1, 0), (0, -1)]))

def which_dir(h, t):
    result = []
    if h[0] == t[0] or h[1] == t[1]:
        if h[0] - t[0] > 0:
            result.append('R')
        elif h[0] - t[0] < 0:
            result.append('L')
        elif h[1] - t[1] > 0:
            result.append('U')
        elif h[1] - t[1] < 0:
            result.append('D')
    else:
        result.append('R') if h[0] > t[0] else result.append('L')
        result.append('U') if h[1] > t[1] else result.append('D')
    return result        
        
def get_prod(h):
    return list(product((h[0]-1, h[0], h[0]+1), (h[1]-1, h[1], h[1]+1)))

In [668]:
visited = {(0, 0)}
length = 10
snake = [(0, 0)] * length
for l in lista:
    d, n = l.split()
    
    for _ in range(int(n)):
        snake[0] = tuple(np.add(snake[0], dirs[d]))
        for h, t in zip(range(0, len(snake)), range(1, len(snake))):
            if snake[t] not in get_prod(snake[h]):
                for w in which_dir(snake[h], snake[t]):
                    snake[t] = tuple(np.add(snake[t], dirs[w]))
            if t == length - 1:
                visited.add(snake[t])

In [669]:
len(visited)

2651

# Day 10

In [275]:
source = open_source('10')
lista = []
for l in source:
    if l.startswith('a'):
        lista.append('noop')
        lista.append(l)
    else:
        lista.append('noop')

In [276]:
strength, cycle, signal_sum, pixel = 1, 0, 0, ''
for l in lista:
    if cycle % 40 - 1 <= strength <= cycle % 40 + 1:
        pixel += "#"
    else:
        pixel += "."
    if cycle + 1 in list(range(20, 221, 40)):
        signal_sum += strength * (cycle + 1)
    if l.startswith('n'):
        cycle += 1
    elif l.startswith('a'):
        strength += int(l.split()[-1])
        cycle += 1
print(signal_sum)
_ = [print(pixel[40 * x:40 * x + 40]) for x in range(6)]

13860
###..####.#..#.####..##....##..##..###..
#..#....#.#..#.#....#..#....#.#..#.#..#.
#..#...#..####.###..#.......#.#....###..
###...#...#..#.#....#.##....#.#....#..#.
#.#..#....#..#.#....#..#.#..#.#..#.#..#.
#..#.####.#..#.#.....###..##...##..###..


# Day 11

In [281]:
source = open_source('11')
source.append('')

lista = defaultdict(lambda: defaultdict(str))
item_status = defaultdict(list)
total_modulo = 1

for monkey, start, op, test, true, false, _ in zip(*[iter(source)]*7):
    d = defaultdict(str)
    monke = monkey.lower().rstrip(':')
    item_status[monke]= list(map(int, start.split(': ')[-1].split(', ')))
    d['op'] = op.split(' = ')[-1]
    d['test'] = test.split(': ')[-1].replace('divisible by ', 'new % ') + ' == 0'
    total_modulo *= int(test.split()[-1])
    d['true'] = true.split('to ')[-1]
    d['false'] = false.split('to ')[-1]
    lista[monke] = d

In [282]:
inspections = defaultdict(int)
for r in range(10000):
    for monkey, a in lista.items():
        for old in item_status[monkey]:
            new = eval(a['op']) % total_modulo # // 3
            if eval(a['test']):
                item_status[a['true']].append(new)
            else:
                item_status[a['false']].append(new)
            inspections[monkey] += 1
        item_status[monkey].clear()

In [283]:
order = sorted(list(inspections.values()))
order[-2] * order[-1]

14106266886

# Day 12

In [96]:
source = open_source('12')
source = np.array([[ord(c) - ord('a') for c in list(x)] for x in source])
sx, sy = list(zip(*np.where(source == -14)))[0]
ex, ey = list(zip(*np.where(source == -28)))[0]
source[source==-14] = 0
source[source==-28] = 25

directions = [(-1, 0), (0, -1), (1, 0), (0, 1)]
lenx, leny = source.shape[0], source.shape[1]

In [104]:
def get_shortest_hike(queue):
    status = defaultdict(int)
    for x, y in queue:
        status[x, y] = 0
    result = []
    num = 0
    while queue:
        cx, cy = queue.popleft()
        if (cx, cy) == (ex, ey):
            result = status[ex, ey]
            break
        for dx, dy in directions:
            nx, ny = cx + dx, cy + dy
            if (
                nx in range(lenx) and
                ny in range(leny) and
                (nx, ny) not in status
                and (source[cx, cy] - source[nx, ny]) > -2
            ):
                queue.append((nx, ny))
                status[nx, ny] = status[cx, cy] + 1
    return result

In [105]:
get_shortest_hike(deque([(sx, sy)]))

449

In [88]:
# source_r = np.zeros_like(source)
# for k, v in zip(np.array(range(0, 26)), np.array(range(25, -1, -1))):
#     source_r[source == k] = v

In [106]:
result = []
for x, y in list(zip(*np.where(source == 0))):
    sub_result = get_shortest_hike(deque([(x, y)]))
    if sub_result:
        result.append(sub_result)
min(result)

443

# Day 13

In [6]:
source = open_source('13')

In [11]:
isint = lambda x: isinstance(x, int)
islist = lambda x: isinstance(x, list)

def check(l, r):
    if isint(l) and isint(r):
        return l - r
    elif islist(l) and islist(r):
        for item in map(check, l, r):
            if item:
                return item
        return check(len(l), len(r))
    elif islist(l) and isint(r):
        return check(l, [r])
    elif isint(l) and islist(r):
        return check([l], r)

In [12]:
part_1 = 0
packets = []
for index, packet in enumerate(zip(source[::3], source[1::3])):
    packet = list(map(eval, packet))
    if check(*packet) < 0:
        part_1 += index + 1
    packets += [*packet]
    
packets += [[[2]], [[6]]]
packets.sort(key=cmp_to_key(check))

part_2 = (packets.index([[2]]) + 1) * (packets.index([[6]]) + 1)

print(part_1, part_2)

6623 23049


# Day 14

In [4]:
def get_area(source, offset, sx, sy, part_1):
    width = set()
    height = set()
    for s in source:
        lista = s.split(' -> ')
        for l in lista:
            lx, ly = map(int, l.split(','))
            width.add(lx)
            height.add(ly)
    cave = np.full((max(width) + offset * 2, max(height)), ' ')
    if part_1:
        cave = np.pad(cave, (1,),  'constant', constant_values='~')
    else:
        cave = np.concatenate([cave, np.full((max(width) + offset * 2, 3), ' ')], axis=1)
        cave[:, max(height) + 2] = '#'
    cave[sx + offset, sy] = '+'
    
    return cave

def get_cave(source, cave, offset):
    for s in source:
        lista = s.split(' -> ')
        for a, b in zip(lista[::], lista[1::]):
            ax, ay = map(int, a.split(','))
            bx, by = map(int, b.split(','))
            ax += offset
            bx += offset
            if ax - bx == 0:
                small, large = min(ay, by), max(ay, by)
                cave[ax, small:large + 1] = '#'
            else:
                small, large = min(ax, bx), max(ax, bx)
                cave[small:large + 1, ay] = '#'
    cave = cave.T
    return cave

In [30]:
def drop_sand(p):
    c = p[0], p[1]
    d = p[0] + 1, p[1]
    l = p[0] + 1, p[1] - 1
    r = p[0] + 1, p[1] + 1
    obs = ('#', 'o')
    if (cave[c] == '~' or 
        cave[c] == '@'):
        cave[c] = '@'
        return True
    elif cave[c] == 'o':
        cave[c] = '@'
    elif all(x in obs for x in [cave[d], cave[l], cave[r]]):
        cave[c] = 'o'
    elif (cave[d] in obs and
          cave[l] == ' '):
        drop_sand(l)
    elif (cave[d] in obs and
          cave[l] in obs and
          cave[r] == ' '):
        drop_sand(r)
    elif cave[c] in ('+', ' '):
        drop_sand(d)

In [37]:
source = open_source('14')

part_1 = False
offset = 0 if part_1 else 300

cave = get_area(source, offset, 500, 0, part_1)
cave = get_cave(source, cave, offset)

i = -1
while '@' not in cave:
    drop_sand(p=(0, 500 + offset))
    part = 1 if part_1 else 2
    i += 1

print(f'part_{part} result: {i}')

y_coords = np.where(cave == 'o')[1]
cutl, cutr = min(y_coords) - 1, max(y_coords) + 2
np.savetxt(f'source/day14_part_{part}_viz.txt', cave[:,cutl:cutr], fmt='%s')

part_2 result: 25248


# Day 15

In [153]:
source = open_source('15')
source

['Sensor at x=2300471, y=2016823: closest beacon is at x=2687171, y=2822745',
 'Sensor at x=1315114, y=37295: closest beacon is at x=1671413, y=43557',
 'Sensor at x=1039523, y=3061589: closest beacon is at x=1570410, y=3710085',
 'Sensor at x=214540, y=3768792: closest beacon is at x=-355567, y=3900317',
 'Sensor at x=1641345, y=3524291: closest beacon is at x=1570410, y=3710085',
 'Sensor at x=1016825, y=1450262: closest beacon is at x=745731, y=2000000',
 'Sensor at x=2768110, y=3703050: closest beacon is at x=3133588, y=3984216',
 'Sensor at x=2213658, y=3522463: closest beacon is at x=1570410, y=3710085',
 'Sensor at x=3842967, y=3381135: closest beacon is at x=3839159, y=3421933',
 'Sensor at x=3952516, y=2683159: closest beacon is at x=3213800, y=2708360',
 'Sensor at x=172892, y=369117: closest beacon is at x=-228964, y=1438805',
 'Sensor at x=3999720, y=3498306: closest beacon is at x=3839159, y=3421933',
 'Sensor at x=1596187, y=307084: closest beacon is at x=1671413, y=43557

In [151]:
row = 2000000
all_occupied = []
to_remove = set()
for x in source:
    sx, sy, bx, by = list(map(int, re.findall(r"-?\d+", x)))
    deniro_dist = abs(sx - bx) + abs(sy - by) - abs(sy - row)
    if deniro_dist >= 0:
        all_occupied.append((sx - deniro_dist, sx + deniro_dist))
    if by == row:
        to_remove.add(bx)

In [152]:
result = set.union(*[set(range(a, b + 1)) for a, b in all_occupied]) - to_remove
len(result)

5838453

In [137]:
# 0 < x , y < 4000000

[(12, 12), (2, 14), (2, 2), (-2, 2), (16, 24), (14, 18)]

In [105]:
X

[(1124672, 3476270),
 (921729, 1157317),
 (745731, 1287919),
 (3871758, 4033274),
 (131435, 745731),
 (1874510, 3922160),
 (2505440, 5493452),
 (745731, 2123201),
 (-345001, 745731)]

In [106]:
[set(range(a, b + 1)) for a, b in X]

[{1124672,
  1124673,
  1124674,
  1124675,
  1124676,
  1124677,
  1124678,
  1124679,
  1124680,
  1124681,
  1124682,
  1124683,
  1124684,
  1124685,
  1124686,
  1124687,
  1124688,
  1124689,
  1124690,
  1124691,
  1124692,
  1124693,
  1124694,
  1124695,
  1124696,
  1124697,
  1124698,
  1124699,
  1124700,
  1124701,
  1124702,
  1124703,
  1124704,
  1124705,
  1124706,
  1124707,
  1124708,
  1124709,
  1124710,
  1124711,
  1124712,
  1124713,
  1124714,
  1124715,
  1124716,
  1124717,
  1124718,
  1124719,
  1124720,
  1124721,
  1124722,
  1124723,
  1124724,
  1124725,
  1124726,
  1124727,
  1124728,
  1124729,
  1124730,
  1124731,
  1124732,
  1124733,
  1124734,
  1124735,
  1124736,
  1124737,
  1124738,
  1124739,
  1124740,
  1124741,
  1124742,
  1124743,
  1124744,
  1124745,
  1124746,
  1124747,
  1124748,
  1124749,
  1124750,
  1124751,
  1124752,
  1124753,
  1124754,
  1124755,
  1124756,
  1124757,
  1124758,
  1124759,
  1124760,
  1124761,
  1124762,

In [87]:
def deniro_circle(cx, cy, r):
    res = set()
    rang = list(range(r + 1))
    for x, y in product(rang, rang):
        if abs(x) + abs(y) == r:
            res.add((cx + x, cy + y))
            res.add((cx - x, cy + y))
            res.add((cx + x, cy - y))
            res.add((cx - x, cy - y))
    return list(res)

In [88]:
deniro_circle(1, 1, 9)

{(-8, 1),
 (-7, 0),
 (-7, 2),
 (-6, -1),
 (-6, 3),
 (-5, -2),
 (-5, 4),
 (-4, -3),
 (-4, 5),
 (-3, -4),
 (-3, 6),
 (-2, -5),
 (-2, 7),
 (-1, -6),
 (-1, 8),
 (0, -7),
 (0, 9),
 (1, -8),
 (1, 10),
 (2, -7),
 (2, 9),
 (3, -6),
 (3, 8),
 (4, -5),
 (4, 7),
 (5, -4),
 (5, 6),
 (6, -3),
 (6, 5),
 (7, -2),
 (7, 4),
 (8, -1),
 (8, 3),
 (9, 0),
 (9, 2),
 (10, 1)}

# Day 16

# Day 17

# Day 18

In [422]:
source = open_source('18')

In [423]:
conns = ((1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1))
cubes = {tuple(map(int, s.split(','))) for s in source}

In [433]:
get_conn = lambda a, b: tuple([sum(x) for x in (zip(cube, conn))])

In [436]:
part_1 = 0
for cube, conn in product(cubes, conns):
    if get_conn(cube, conn) not in cubes:
        part_1 += 1
part_1

4604

In [425]:
mx = max([max(t[0], t[1], t[2]) for t in cubes]) + 2
mn = min([min(t[0], t[1], t[2]) for t in cubes]) - 1

In [437]:
todo = [(mn, mn, mn)]
seen = set()
part_2 = 0
while todo:
    cube = todo.pop()
    for conn in conns:
        c = get_conn(cube, conn)
        if all([mn <= x <= mx for x in c]):
            if c in cubes:
                part_2 += 1
            elif c not in seen:
                todo.append(c)
            seen.add(c)
part_2

2604

# Day 19

In [111]:
source = open_source('19_example')
source 

['Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.',
 'Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.']

In [33]:
blueprints = defaultdict(dict)
for s in source:
    bp, ore, clay, obs1, obs2, geo1, geo2 = list(map(int, re.findall(r"-?\d+", s)))
    blueprint = {}
    blueprint['or'] = (('o', ore))
    blueprint['cr'] = (('o', clay))
    blueprint['br'] = (('o', obs1), ('c', obs2))
    blueprint['gr'] = (('o', geo1), ('b', geo2))
    blueprints[int(bp)] = blueprint

In [34]:
pp.pprint(blueprints[1])

{ 'br': (('o', 3), ('c', 14)),
  'cr': ('o', 2),
  'gr': (('o', 2), ('b', 7)),
  'or': ('o', 4)}


In [69]:
resources = {
    'o': 0,
    'c': 0,
    'b': 0,
    'g': 0
}
robots = {
    'or': 1,
    'cr': 0,
    'br': 0,
    'gr': 0,
}

robot_types = ['or', 'cr', 'br', 'gr', 'no']

def mine(res, rob):
    for k, v in rob.items():
        res[k[0]] += v
    return res
        
def build(res, rob, bp, rt):
    if not rt == 'no':
        if all(v <= res[k] for k, v in bp[rt].items()):
            for k, v in bp[rt].items():
                res[k] -= v
            rob[rt] += 1
    return res, rob

@freezeargs
@lru_cache
def build_mine(res, rob, bp, rt, minute):
    rob_prev = deepcopy(rob)
    res = dict(res)
    rob = dict(rob)
    if not rt == 'no':
        if all(y <= res[x] for x, y in zip(bp['or'][::2], bp['or'][1::2])):
            for x, y in zip(bp['or'][::2], bp['or'][1::2]):
                res[x] -= y
            rob[rt] += 1
    for k, v in rob_prev.items():
        res[k[0]] += v
    if minute == 0:
        all_results.add(res['g'])
        return True
    else:
        for t in robot_types:
            build_mine(res, rob, bp, rt, minute - 1)

In [651]:
geode_max = 0
for i in range(100_000):
    resources = {
    'o': 0,
    'c': 0,
    'b': 0,
    'g': 0
    }
    robots = {
    'or': 1,
    'cr': 0,
    'br': 0,
    'gr': 0,
    }
    for j in range(24):
        resources, robots = build_mine(resources, robots, blueprints[1], random.choice(['or', 'cr', 'br', 'gr', 'no']))    
    if geode_max < resources['g']:
        geode_max = resources['g']
geode_max

8

# Day 20

# Day 21

# Day 22

# Day 23

# Day 24

# Day 25